In [1]:
# Libraries
import os
import glob
import pandas as pd
import numpy as np
import csv
from IPython.display import display, HTML
import requests
import urllib.request
import os

# Suppressing warning to make notebook more public ready
import warnings
warnings.filterwarnings("ignore")

In [2]:
# File locations
loc_book_finished_2024 = r'YOUR_PATH\Kindle\Kindle.ReadingInsights\datasets\Kindle.UserUniqueTitlesCompleted\Kindle.UserUniqueTitlesCompleted.csv'
loc_book_time_read = r'YOUR_PATH\Kindle.ReadingInsights\datasets\Kindle.reading-insights-sessions_with_adjustments\Kindle.reading-insights-sessions_with_adjustments.csv'
loc_book_author_name = r'YOUR_PATH\Kindle.UnifiedLibraryIndex\datasets\Kindle.UnifiedLibraryIndex.CustomerAuthorNameRelationship\Kindle.UnifiedLibraryIndex.CustomerAuthorNameRelationship.csv'
loc_book_genre = r'YOUR_PATH\Kindle\Kindle\Kindle.UnifiedLibraryIndex\datasets\Kindle.UnifiedLibraryIndex.CustomerGenres\Kindle.UnifiedLibraryIndex.CustomerGenres.csv'

In [3]:
#Books read in 2024
import datetime
df_book_finished_2024 = pd.read_csv(loc_book_finished_2024)
df_book_finished_2024 = df_book_finished_2024.drop_duplicates().reset_index(drop = True)
df_book_finished_2024[['asin', 'date_finished', 'finish_type']] = df_book_finished_2024['asin_date_and_content_type'].str.split('_', expand=True)
df_book_finished_2024['date_finished'] = pd.to_datetime(df_book_finished_2024['date_finished'])
df_book_finished_2024 = df_book_finished_2024[(df_book_finished_2024['date_finished'] >= datetime.datetime(2024, 1, 1))]
# Manual updates
df_book_finished_2024.loc[len(df_book_finished_2024)] = [None,"B08Q42W59Z",datetime.datetime(2024, 12, 15),None]
df_book_finished_2024.loc[len(df_book_finished_2024)] = [None,"B0CTXPG5QS",datetime.datetime(2024, 11, 23),None]
df_book_finished_2024 = df_book_finished_2024.sort_values('date_finished').reset_index(drop=True)
print(df_book_finished_2024.head(20))

         asin_date_and_content_type        asin date_finished finish_type
0      B002361MLA_2024-01-25_MANUAL  B002361MLA    2024-01-25      MANUAL
1      B08CD1M43L_2024-01-25_MANUAL  B08CD1M43L    2024-01-25      MANUAL
2      B002361MLA_2024-02-13_MANUAL  B002361MLA    2024-02-13      MANUAL
3   B003T0G9E4_2024-03-03_AUTOMATIC  B003T0G9E4    2024-03-03   AUTOMATIC
4   B07D61RYYV_2024-03-23_AUTOMATIC  B07D61RYYV    2024-03-23   AUTOMATIC
5      B07MX3YB51_2024-04-04_MANUAL  B07MX3YB51    2024-04-04      MANUAL
6   B0BTQ4GG2F_2024-06-29_AUTOMATIC  B0BTQ4GG2F    2024-06-29   AUTOMATIC
7      B07MJ3TDCZ_2024-07-02_MANUAL  B07MJ3TDCZ    2024-07-02      MANUAL
8      B00KEPLBYK_2024-07-06_MANUAL  B00KEPLBYK    2024-07-06      MANUAL
9      B07FB1NPNY_2024-09-21_MANUAL  B07FB1NPNY    2024-09-21      MANUAL
10     B01F1UD66S_2024-10-09_MANUAL  B01F1UD66S    2024-10-09      MANUAL
11                             None  B0CTXPG5QS    2024-11-23        None
12                             None  B

In [4]:
# Reading Summary
import datetime
df_book_time_read = pd.read_csv(loc_book_time_read)
df_book_time_read = df_book_time_read.drop_duplicates().reset_index(drop = True)
df_book_time_read['start_time'] = pd.to_datetime(df_book_time_read['start_time']).dt.date
df_book_time_read = df_book_time_read[(df_book_time_read['start_time'] >= datetime.date(2023, 1, 1))]
df_book_time_read['end_time'] = pd.to_datetime(df_book_time_read['end_time']).dt.date
df_book_time_read = df_book_time_read.groupby(['ASIN'])
df_book_time_read = df_book_time_read.agg(min_start_date=('start_time', np.min), 
                                          max_end_date=('end_time', np.max), 
                                          read_time_ms=('total_reading_milliseconds', np.sum))
df_book_time_read['days_to_read'] = df_book_time_read['max_end_date'] - df_book_time_read['min_start_date']
df_book_time_read = df_book_time_read.sort_values('max_end_date')
print(df_book_time_read.tail(20))

           min_start_date max_end_date  read_time_ms days_to_read
ASIN                                                             
B08CD1M43L     2023-11-23   2024-01-25      31776000      63 days
B002361MLA     2023-12-03   2024-02-13      18307900      72 days
B003T0G9E4     2024-02-05   2024-03-03      15621000      27 days
B07D61RYYV     2024-02-02   2024-03-23      21378600      50 days
B00CD428BU     2023-08-07   2024-04-02     106792100     239 days
B07MX3YB51     2023-11-19   2024-04-04      42355400     137 days
B097824MVJ     2024-04-06   2024-04-06       1812900       0 days
B0BTQ4GG2F     2024-04-20   2024-06-29      11123600      70 days
B00KEPLBYK     2024-03-04   2024-07-06      85522000     124 days
B07FB1NPNY     2024-07-12   2024-09-21      46390000      71 days
B01F1UD66S     2024-07-06   2024-10-10      38252200      96 days
B004H4XI5O     2024-10-16   2024-10-16         12400       0 days
B0CTXPG5QS     2024-10-05   2024-10-16      52442400      11 days
B007Q6XLF2

In [5]:
# Book info
df_book_author_name = pd.read_csv(loc_book_author_name)
df_book_author_name = df_book_author_name.drop_duplicates(subset=['ASIN']).reset_index(drop = True)
df_book_author_name = pd.DataFrame(df_book_author_name)
display(HTML(df_book_author_name.to_html()))

,Product Name,ASIN,Author Name
0,"Snowflake: The Definitive Guide: Architecting, Designing, and Deploying on the Snowflake Data Cloud",1098103823,"Avila, Joyce Kay"
1,Rise of the Data Cloud,B08Q42W59Z,"Slootman, Frank"
2,"AI Snake Oil: What Artificial Intelligence Can Do, What It Can’t, and How to Tell the Difference",B0CW1JCKVL,"Narayanan, Arvind"
3,"The Why Is Everything: A Story of Football, Rivalry, and Revolution",B0CTXPG5QS,"Silver, Michael"
4,"Built from Broken: A Science-Based Guide to Healing Painful Joints, Preventing Injuries, and Rebuilding Your Body",1735728500,"Hogan, Scott H"
5,A Peace to End All Peace: The Fall of the Ottoman Empire and the Creation of the Modern Middle East,B003X27L7C,"Fromkin, David"
6,Moonwalking with Einstein: The Art and Science of Remembering Everything,B004H4XI5O,"Foer, Joshua"
7,Mindset: The New Psychology of Success,B000FCKPHG,"Dweck, Carol S."
8,Naked Economics: Undressing the Dismal Science,B07FB1NPNY,"Wheelan, Charles J."
9,"The Telomere Effect: A Revolutionary Approach to Living Younger, Healthier, Longer",B01F1UD66S,"Blackburn, Dr. Elizabeth"


In [45]:
# Book genres
asin_filter = df_book_finished_2024['asin'].tolist()
df_book_genre = pd.read_csv(loc_book_genre)
df_book_genre['asin_str'] = df_book_genre['ASIN'].apply(''.join)
df_book_genre = df_book_genre[df_book_genre['asin_str'].isin(asin_filter)]
df_book_genre = df_book_genre.drop_duplicates().reset_index(drop = True)

print(df_book_genre.head(10))

                                        Product Name        ASIN  \
0                             Rise of the Data Cloud  B08Q42W59Z   
1  The Why Is Everything: A Story of Football, Ri...  B0CTXPG5QS   
2  The Why Is Everything: A Story of Football, Ri...  B0CTXPG5QS   
3     Naked Economics: Undressing the Dismal Science  B07FB1NPNY   
4  The Telomere Effect: A Revolutionary Approach ...  B01F1UD66S   
5  The Telomere Effect: A Revolutionary Approach ...  B01F1UD66S   
6                               Stillness Is the Key  B07MJ3TDCZ   
7                               Stillness Is the Key  B07MJ3TDCZ   
8                               Stillness Is the Key  B07MJ3TDCZ   
9  The Year in Tech, 2024: The Insights You Need ...  B0BTQ4GG2F   

                        Genre    asin_str  
0            Business & Money  B08Q42W59Z  
1           Sports & Outdoors  B0CTXPG5QS  
2       Biographies & Memoirs  B0CTXPG5QS  
3            Business & Money  B07FB1NPNY  
4                   Self-Help  

In [47]:
# Combine kindle data sets
df = df_book_finished_2024
df = pd.merge(df, df_book_time_read, 
              left_on=['asin'], 
              right_on=['ASIN'],
              how='left')
df = pd.merge(df, df_book_author_name, 
              left_on=['asin'], 
              right_on=['ASIN'],
              how='left')

def replace_start_the(text, old, new):
    if text.startswith(old):
        return new + text[len(old):]
    return text

# Create values for api calls
df['search_title'] = df['Product Name'].str.lower()
df['search_title'] = df['search_title'].str.partition(":")[0]
df['search_title'] = df['search_title'].str.partition("(")[0].str.strip()
df['search_title'] = df['search_title'].str.replace("'","")
df['search_title'] = df['search_title'].str.replace(",","")
df['search_title'] = df['search_title'].apply(lambda x: replace_start_the(x,"the ",""))
df['search_title'] = df['search_title'].str.replace(" ","+")

df['search_author'] = df['Author Name'].str.lower()
df['search_author'] = df['search_author'].str.partition(",")[0]
# Clean up duplicate
df.drop(2).reset_index(drop = True)

,asin_date_and_content_type,asin,date_finished,finish_type,min_start_date,max_end_date,read_time_ms,days_to_read,Product Name,ASIN,Author Name,search_title,search_author
0,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,"Tolle, Eckhart",power+of+now,tolle
1,B08CD1M43L_2024-01-25_MANUAL,B08CD1M43L,2024-01-25,MANUAL,2023-11-23,2024-01-25,31776000,63 days,Genius Makers: The Mavericks Who Brought AI to...,B08CD1M43L,"Metz, Cade",genius+makers,metz
2,B003T0G9E4_2024-03-03_AUTOMATIC,B003T0G9E4,2024-03-03,AUTOMATIC,2024-02-05,2024-03-03,15621000,27 days,The Inner Game of Tennis: The Classic Guide to...,B003T0G9E4,"Gallwey, W. Timothy",inner+game+of+tennis,gallwey
3,B07D61RYYV_2024-03-23_AUTOMATIC,B07D61RYYV,2024-03-23,AUTOMATIC,2024-02-02,2024-03-23,21378600,50 days,"HBR's 10 Must Reads on AI, Analytics, and the ...",B07D61RYYV,Harvard Business Review,hbrs+10+must+reads+on+ai+analytics+and+the+new...,harvard business review
4,B07MX3YB51_2024-04-04_MANUAL,B07MX3YB51,2024-04-04,MANUAL,2023-11-19,2024-04-04,42355400,137 days,American Bonds: How Credit Markets Shaped a Na...,B07MX3YB51,"Quinn, Sarah L.",american+bonds,quinn
5,B0BTQ4GG2F_2024-06-29_AUTOMATIC,B0BTQ4GG2F,2024-06-29,AUTOMATIC,2024-04-20,2024-06-29,11123600,70 days,"The Year in Tech, 2024: The Insights You Need ...",B0BTQ4GG2F,"Review, Harvard Business",year+in+tech+2024,review
6,B07MJ3TDCZ_2024-07-02_MANUAL,B07MJ3TDCZ,2024-07-02,MANUAL,2024-04-23,2024-10-18,24934800,178 days,Stillness Is the Key,B07MJ3TDCZ,"Holiday, Ryan",stillness+is+the+key,holiday
7,B00KEPLBYK_2024-07-06_MANUAL,B00KEPLBYK,2024-07-06,MANUAL,2024-03-04,2024-07-06,85522000,124 days,Frank Lloyd Wright,B00KEPLBYK,"Secrest, Meryle",frank+lloyd+wright,secrest
8,B07FB1NPNY_2024-09-21_MANUAL,B07FB1NPNY,2024-09-21,MANUAL,2024-07-12,2024-09-21,46390000,71 days,Naked Economics: Undressing the Dismal Science,B07FB1NPNY,"Wheelan, Charles J.",naked+economics,wheelan
9,B01F1UD66S_2024-10-09_MANUAL,B01F1UD66S,2024-10-09,MANUAL,2024-07-06,2024-10-10,38252200,96 days,The Telomere Effect: A Revolutionary Approach ...,B01F1UD66S,"Blackburn, Dr. Elizabeth",telomere+effect,blackburn


In [8]:
api_url = r'https://openlibrary.org/search.json?title='
api_title = df['search_title'].tolist()
df_api_result = pd.DataFrame()
search_title_value = []

for i in api_title:
    rm = requests.get(api_url+i)
    search_title_value.append(i)
    df_api_result = pd.concat([df_api_result,pd.json_normalize(rm.json()['docs'])])
    
df_api_result

,author_alternative_name,author_key,author_name,contributor,cover_edition_key,cover_i,ddc,ebook_access,ebook_count_i,edition_count,...,id_better_world_books,place,place_key,place_facet,person,person_key,person_facet,time,time_facet,time_key
0,"[Tolle, Eckhart, 1948-]",[OL1392686A],[Eckhart Tolle],"[Cao, Zhi.]",OL8154493M,551262.0,[291.44],borrowable,5,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Tolle, Eckhart, 1948-]",[OL1392686A],[Eckhart Tolle],NaN,OL24946036M,6878445.0,"[204.4, 291.44]",borrowable,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Tolle, Eckhart, 1948-]",[OL1392686A],[Eckhart Tolle],"[Tolle, Eckhart, 1948-]",OL3303618M,1932143.0,"[291.44, 204.4]",borrowable,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Tolle, Eckhart, 1948-]",[OL1392686A],[Eckhart Tolle],NaN,OL24930694M,6874340.0,[158.1],borrowable,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Tolle, Eckhart, 1948-]",[OL1392686A],[Eckhart Tolle],NaN,OL10679488M,7024931.0,NaN,no_ebook,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,"[Edward Vernon, Vernon Dr Coleman, Dr. Vernon ...",[OL977413A],[Vernon Coleman],NaN,OL16233233M,11958349.0,NaN,borrowable,1,1,...,NaN,[Great Britain],[great_britain],[Great Britain],NaN,NaN,NaN,[21st century],[21st century],[21st_century]
19,NaN,[OL10478572A],[Joseph Nguyen],NaN,NaN,NaN,NaN,no_ebook,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,[OL10478572A],[Joseph Nguyen],NaN,OL57447337M,14833784.0,NaN,no_ebook,0,1,...,NaN,NaN,NaN,NaN,"[Kenna(his love), many other respected and wel...","[kenna(his_love), lao_tzu_and_etc., many_other...","[Kenna(his love), Lao Tzu and etc., Steve Jobs...",NaN,NaN,NaN
21,NaN,[OL8515224A],[Ali Fenwick],NaN,NaN,NaN,NaN,no_ebook,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_api_clean = df_api_result[['key','author_name','cover_edition_key','cover_i','title','number_of_pages_median',
                             'first_publish_year']]

df_api_clean['search_title'] = df_api_clean['title'].str.lower()
df_api_clean['search_title'] = df_api_clean['search_title'].str.partition(":")[0]
df_api_clean['search_title'] = df_api_clean['search_title'].str.partition("(")[0].str.strip()
df_api_clean['search_title'] = df_api_clean['search_title'].str.replace("'","")
df_api_clean['search_title'] = df_api_clean['search_title'].str.replace(",","")
df_api_clean['search_title'] = df_api_clean['search_title'].apply(lambda x: replace_start_the(x,"the ",""))
df_api_clean['search_title'] = df_api_clean['search_title'].str.replace(" ","+")

df_api_clean

,key,author_name,cover_edition_key,cover_i,title,number_of_pages_median,first_publish_year,search_title
0,/works/OL5727686W,[Eckhart Tolle],OL8154493M,551262.0,The Power of Now,201.0,1997.0,power+of+now
1,/works/OL16045198W,[Eckhart Tolle],OL24946036M,6878445.0,The power of now,235.0,2004.0,power+of+now
2,/works/OL5727684W,[Eckhart Tolle],OL3303618M,1932143.0,Practicing the Power of Now,142.0,1999.0,practicing+the+power+of+now
3,/works/OL15688311W,[Eckhart Tolle],OL24930694M,6874340.0,Practising the power of now,134.0,2002.0,practising+the+power+of+now
4,/works/OL9465675W,[Eckhart Tolle],OL10679488M,7024931.0,Power of Now,224.0,2005.0,power+of+now
...,...,...,...,...,...,...,...,...
18,/works/OL11293353W,[Vernon Coleman],OL16233233M,11958349.0,Why everything is going to get worse before it...,319.0,2004.0,why+everything+is+going+to+get+worse+before+it...
19,/works/OL42346043W,[Joseph Nguyen],NaN,NaN,Don't Believe Everything You Think Why Your Th...,NaN,2022.0,dont+believe+everything+you+think+why+your+thi...
20,/works/OL42346044W,[Joseph Nguyen],OL57447337M,14833784.0,Don't Believe Everything You Think Why Your Th...,87.0,2022.0,dont+believe+everything+you+think+why+your+thi...
21,/works/OL30748665W,[Ali Fenwick],NaN,NaN,Everything You Need to Know about Clubhouse : ...,NaN,2021.0,everything+you+need+to+know+about+clubhouse


In [48]:
df = pd.merge(df, df_api_clean, 
              left_on=['search_title'], 
              right_on=['search_title'],
              how='left')

df

,asin_date_and_content_type,asin,date_finished,finish_type,min_start_date,max_end_date,read_time_ms,days_to_read,Product Name,ASIN,Author Name,search_title,search_author,key,author_name,cover_edition_key,cover_i,title,number_of_pages_median,first_publish_year
0,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,"Tolle, Eckhart",power+of+now,tolle,/works/OL5727686W,[Eckhart Tolle],OL8154493M,551262.0,The Power of Now,201.0,1997.0
1,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,"Tolle, Eckhart",power+of+now,tolle,/works/OL16045198W,[Eckhart Tolle],OL24946036M,6878445.0,The power of now,235.0,2004.0
2,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,"Tolle, Eckhart",power+of+now,tolle,/works/OL9465675W,[Eckhart Tolle],OL10679488M,7024931.0,Power of Now,224.0,2005.0
3,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,"Tolle, Eckhart",power+of+now,tolle,/works/OL7986372W,[Eckhart Tolle],OL10629248M,203917.0,THE POWER OF NOW,224.0,2001.0
4,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,"Tolle, Eckhart",power+of+now,tolle,/works/OL29253293W,[Eckhart Tolle],OL44281743M,13345056.0,The Power of Now,287.0,2006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,B01F1UD66S_2024-10-09_MANUAL,B01F1UD66S,2024-10-09,MANUAL,2024-07-06,2024-10-10,38252200,96 days,The Telomere Effect: A Revolutionary Approach ...,B01F1UD66S,"Blackburn, Dr. Elizabeth",telomere+effect,blackburn,/works/OL20043578W,[Elizabeth H. Blackburn],NaN,NaN,The telomere effect,398.0,2017.0
215,B01F1UD66S_2024-10-09_MANUAL,B01F1UD66S,2024-10-09,MANUAL,2024-07-06,2024-10-10,38252200,96 days,The Telomere Effect: A Revolutionary Approach ...,B01F1UD66S,"Blackburn, Dr. Elizabeth",telomere+effect,blackburn,/works/OL20046004W,[Elizabeth H. Blackburn],NaN,NaN,The telomere effect : (large print),520.0,2017.0
216,B01F1UD66S_2024-10-09_MANUAL,B01F1UD66S,2024-10-09,MANUAL,2024-07-06,2024-10-10,38252200,96 days,The Telomere Effect: A Revolutionary Approach ...,B01F1UD66S,"Blackburn, Dr. Elizabeth",telomere+effect,blackburn,/works/OL20209843W,"[Dr Elizabeth Blackburn, Dr Elissa Epel]",OL27398141M,8860550.0,The Telomere Effect : A Revolutionary approach...,NaN,2017.0
217,None,B0CTXPG5QS,2024-11-23,None,2024-10-05,2024-10-16,52442400,11 days,"The Why Is Everything: A Story of Football, Ri...",B0CTXPG5QS,"Silver, Michael",why+is+everything,silver,/works/OL37877413W,[Michael Silver],OL51131536M,14622316.0,Why Is Everything,NaN,2024.0


In [53]:
open_lib_url = 'https://openlibrary.org'
def author_contains(text, y):
    if text.find(y) > 0:
        return True
    return False

def has_page_num(n):
    if n > 0:
        return True
    return False

df['open_library_link'] = open_lib_url + df['key']
df['author_name_list_string'] = df['author_name'].astype(str).str.lower()
df['author_match'] = df.apply(lambda x: author_contains(x.author_name_list_string, x.search_author), axis=1)
df['has_pages'] = df.apply(lambda x: has_page_num(x.number_of_pages_median), axis=1)
df = df[df['author_match'] == True]
df

,asin_date_and_content_type,asin,date_finished,finish_type,min_start_date,max_end_date,read_time_ms,days_to_read,Product Name,ASIN,...,cover_edition_key,cover_i,title,number_of_pages_median,first_publish_year,author_name_list_string,author_match,has_pages,rn,open_library_link
0,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,...,OL44281743M,13345056.0,The Power of Now,287.0,2006.0,['eckhart tolle'],True,True,1,https://openlibrary.org/works/OL29253293W
1,B08CD1M43L_2024-01-25_MANUAL,B08CD1M43L,2024-01-25,MANUAL,2023-11-23,2024-01-25,31776000,63 days,Genius Makers: The Mavericks Who Brought AI to...,B08CD1M43L,...,OL32042574M,10708874.0,Genius Makers,360.0,2021.0,['cade metz'],True,True,1,https://openlibrary.org/works/OL24233014W
2,B003T0G9E4_2024-03-03_AUTOMATIC,B003T0G9E4,2024-03-03,AUTOMATIC,2024-02-05,2024-03-03,15621000,27 days,The Inner Game of Tennis: The Classic Guide to...,B003T0G9E4,...,OL18344414M,5413630.0,The Inner Game of Tennis,141.0,1834.0,['w. timothy gallwey'],True,True,1,https://openlibrary.org/works/OL2619508W
3,B07D61RYYV_2024-03-23_AUTOMATIC,B07D61RYYV,2024-03-23,AUTOMATIC,2024-02-02,2024-03-23,21378600,50 days,"HBR's 10 Must Reads on AI, Analytics, and the ...",B07D61RYYV,...,OL27334241M,8798891.0,"HBR's 10 Must Reads on AI, Analytics, and the ...",192.0,2019.0,"['harvard business review', 'michael e. porter...",True,True,1,https://openlibrary.org/works/OL20153843W
4,B07MX3YB51_2024-04-04_MANUAL,B07MX3YB51,2024-04-04,MANUAL,2023-11-19,2024-04-04,42355400,137 days,American Bonds: How Credit Markets Shaped a Na...,B07MX3YB51,...,NaN,NaN,American Bonds,312.0,2019.0,['sarah l. quinn'],True,True,1,https://openlibrary.org/works/OL21639864W
5,B0BTQ4GG2F_2024-06-29_AUTOMATIC,B0BTQ4GG2F,2024-06-29,AUTOMATIC,2024-04-20,2024-06-29,11123600,70 days,"The Year in Tech, 2024: The Insights You Need ...",B0BTQ4GG2F,...,NaN,NaN,"Year in Tech, 2024",176.0,2023.0,"['harvard business review', 'david de cremer',...",True,True,1,https://openlibrary.org/works/OL36455510W
6,B07MJ3TDCZ_2024-07-02_MANUAL,B07MJ3TDCZ,2024-07-02,MANUAL,2024-04-23,2024-10-18,24934800,178 days,Stillness Is the Key,B07MJ3TDCZ,...,OL27691631M,9130310.0,Stillness Is the Key,288.0,2019.0,['ryan holiday'],True,True,1,https://openlibrary.org/works/OL20464740W
7,B00KEPLBYK_2024-07-06_MANUAL,B00KEPLBYK,2024-07-06,MANUAL,2024-03-04,2024-07-06,85522000,124 days,Frank Lloyd Wright,B00KEPLBYK,...,OL1744276M,4009164.0,Frank Lloyd Wright,634.0,1992.0,['meryle secrest'],True,True,1,https://openlibrary.org/works/OL1858540W
8,B07FB1NPNY_2024-09-21_MANUAL,B07FB1NPNY,2024-09-21,MANUAL,2024-07-12,2024-09-21,46390000,71 days,Naked Economics: Undressing the Dismal Science,B07FB1NPNY,...,OL24490598M,8233283.0,Naked economics,354.0,2003.0,['charles j. wheelan'],True,True,1,https://openlibrary.org/works/OL15533535W
9,B01F1UD66S_2024-10-09_MANUAL,B01F1UD66S,2024-10-09,MANUAL,2024-07-06,2024-10-10,38252200,96 days,The Telomere Effect: A Revolutionary Approach ...,B01F1UD66S,...,NaN,NaN,The telomere effect : (large print),520.0,2017.0,['elizabeth h. blackburn'],True,True,1,https://openlibrary.org/works/OL20046004W


In [50]:
df['rn'] = df.sort_values(['has_pages','first_publish_year', 'number_of_pages_median'], ascending=[False, False, False])\
    .groupby(['asin'])\
    .cumcount() + 1

df = df[df['rn'] == 1].reset_index(drop = True)
df

,asin_date_and_content_type,asin,date_finished,finish_type,min_start_date,max_end_date,read_time_ms,days_to_read,Product Name,ASIN,...,author_name,cover_edition_key,cover_i,title,number_of_pages_median,first_publish_year,author_name_list_string,author_match,has_pages,rn
0,B002361MLA_2024-01-25_MANUAL,B002361MLA,2024-01-25,MANUAL,2023-12-03,2024-02-13,18307900,72 days,The Power of Now: A Guide to Spiritual Enlight...,B002361MLA,...,[Eckhart Tolle],OL44281743M,13345056.0,The Power of Now,287.0,2006.0,['eckhart tolle'],True,True,1
1,B08CD1M43L_2024-01-25_MANUAL,B08CD1M43L,2024-01-25,MANUAL,2023-11-23,2024-01-25,31776000,63 days,Genius Makers: The Mavericks Who Brought AI to...,B08CD1M43L,...,[Cade Metz],OL32042574M,10708874.0,Genius Makers,360.0,2021.0,['cade metz'],True,True,1
2,B003T0G9E4_2024-03-03_AUTOMATIC,B003T0G9E4,2024-03-03,AUTOMATIC,2024-02-05,2024-03-03,15621000,27 days,The Inner Game of Tennis: The Classic Guide to...,B003T0G9E4,...,[W. Timothy Gallwey],OL18344414M,5413630.0,The Inner Game of Tennis,141.0,1834.0,['w. timothy gallwey'],True,True,1
3,B07D61RYYV_2024-03-23_AUTOMATIC,B07D61RYYV,2024-03-23,AUTOMATIC,2024-02-02,2024-03-23,21378600,50 days,"HBR's 10 Must Reads on AI, Analytics, and the ...",B07D61RYYV,...,"[Harvard Business Review, Michael E. Porter, T...",OL27334241M,8798891.0,"HBR's 10 Must Reads on AI, Analytics, and the ...",192.0,2019.0,"['harvard business review', 'michael e. porter...",True,True,1
4,B07MX3YB51_2024-04-04_MANUAL,B07MX3YB51,2024-04-04,MANUAL,2023-11-19,2024-04-04,42355400,137 days,American Bonds: How Credit Markets Shaped a Na...,B07MX3YB51,...,[Sarah L. Quinn],NaN,NaN,American Bonds,312.0,2019.0,['sarah l. quinn'],True,True,1
5,B0BTQ4GG2F_2024-06-29_AUTOMATIC,B0BTQ4GG2F,2024-06-29,AUTOMATIC,2024-04-20,2024-06-29,11123600,70 days,"The Year in Tech, 2024: The Insights You Need ...",B0BTQ4GG2F,...,"[Harvard Business Review, David De Cremer, Ric...",NaN,NaN,"Year in Tech, 2024",176.0,2023.0,"['harvard business review', 'david de cremer',...",True,True,1
6,B07MJ3TDCZ_2024-07-02_MANUAL,B07MJ3TDCZ,2024-07-02,MANUAL,2024-04-23,2024-10-18,24934800,178 days,Stillness Is the Key,B07MJ3TDCZ,...,[Ryan Holiday],OL27691631M,9130310.0,Stillness Is the Key,288.0,2019.0,['ryan holiday'],True,True,1
7,B00KEPLBYK_2024-07-06_MANUAL,B00KEPLBYK,2024-07-06,MANUAL,2024-03-04,2024-07-06,85522000,124 days,Frank Lloyd Wright,B00KEPLBYK,...,[Meryle Secrest],OL1744276M,4009164.0,Frank Lloyd Wright,634.0,1992.0,['meryle secrest'],True,True,1
8,B07FB1NPNY_2024-09-21_MANUAL,B07FB1NPNY,2024-09-21,MANUAL,2024-07-12,2024-09-21,46390000,71 days,Naked Economics: Undressing the Dismal Science,B07FB1NPNY,...,[Charles J. Wheelan],OL24490598M,8233283.0,Naked economics,354.0,2003.0,['charles j. wheelan'],True,True,1
9,B01F1UD66S_2024-10-09_MANUAL,B01F1UD66S,2024-10-09,MANUAL,2024-07-06,2024-10-10,38252200,96 days,The Telomere Effect: A Revolutionary Approach ...,B01F1UD66S,...,[Elizabeth H. Blackburn],NaN,NaN,The telomere effect : (large print),520.0,2017.0,['elizabeth h. blackburn'],True,True,1


In [77]:
cover_api_url = 'https://covers.openlibrary.org/b/id/'
cover_api_size = '-M'
cover_api_file_type = '.jpg'
cover_api_list = df['cover_i'].dropna().to_list()
image_path = r'C:\YOUR_PATH\\'

for i in cover_api_list:
    name_row = df[df['cover_i']==i]
    cover_title = name_row.iloc[0]['search_title']
    image_url = cover_api_url+str(int(i))+cover_api_size+cover_api_file_type
    urllib.request.urlretrieve(image_url, image_path+cover_title+str(int(i))+cover_api_file_type)
    #print(cover_title)
    #print(cover_api_url+str(int(i))+cover_api_size+cover_api_file_type)

In [46]:
# Save data sets as excel wkbks
save_excel_path = r'YOUR_PATH\\'
save_main = 'kindle_books_read_2024.xlsx'
save_genres = 'kindle_books_read_genres_2024.xlsx'

df.to_excel(save_excel_path+save_main, index=False, header=True)
df_book_genre.to_excel(save_excel_path+save_genres, index=False, header=True)